In [ ]:
import esda
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from libpysal import graph

In [ ]:
gdf = gpd.read_parquet("/data/processed_data/fbg_rgn.parquet")

In [ ]:
gdf

In [ ]:
gdf["% Other"] = gdf["% Other"] * 100

remove nan values to calculate Moran's I

In [ ]:
not_nan = ~np.isnan(gdf["% Other"])

In [ ]:
not_nan

In [ ]:
gdf = gdf[not_nan]

In [ ]:
gdf

In [ ]:
gdf.explore(
    "% Other",
    cmap="coolwarm",
    vmin=0,
    vmax=100,
    prefer_canvas=True,
    tiles="CartoDB Positron",
)

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))
gdf.plot(ax=ax, column="% Other", cmap="coolwarm", vmin=0, vmax=100)

In [ ]:
contiguity = graph.Graph.build_contiguity(gdf, rook=False)

In [ ]:
contiguity

In [ ]:
contiguity_r = contiguity.transform("r")

In [ ]:
gdf["lag"] = contiguity_r.lag(gdf["% Other"])
gdf.head()

In [ ]:
gdf["%O_std"] = (gdf["% Other"] - gdf["% Other"].mean()) / gdf["% Other"].std()

In [ ]:
gdf

In [ ]:
gdf["%O_std_lag"] = contiguity_r.lag(gdf["%O_std"])

In [ ]:
gdf

In [ ]:
f, ax = plt.subplots(1, figsize=(6, 6))
sns.regplot(
    x="%O_std",
    y="%O_std_lag",
    data=gdf,
    marker=".",
    scatter_kws={"alpha": 0.2},
    line_kws=dict(color="lightcoral"),
)
plt.axvline(0, c="black", alpha=0.5)
plt.axhline(0, c="black", alpha=0.5)
plt.text(1.3, 0.5, "High-high", fontsize=10)
plt.text(1, -2.7, "High-low", fontsize=10)
plt.text(-5.4, 0.5, "Low-high", fontsize=10)
plt.text(-5.4, -2.7, "Low-low", fontsize=10);

In [ ]:
f, ax = plt.subplots(1, figsize=(6, 6))
sns.regplot(
    x="% Other",
    y="lag",
    data=gdf,
    marker=".",
    scatter_kws={"alpha": 0.2},
    line_kws=dict(color="lightcoral"),
)
plt.axvline(0, c="black", alpha=0.5)
plt.axhline(0, c="black", alpha=0.5);

In [ ]:
mi = esda.Moran(gdf["% Other"], contiguity_r.to_W())

In [ ]:
mi.I

In [ ]:
print(f"Moran's I: {mi.I}, p-value: {mi.p_sim}")

In [ ]:
lisa = esda.Moran_Local(gdf["% Other"], contiguity_r.to_W())

# Local Spatial Autocorrelation

## 0.1 Significance

In [ ]:
gdf_01 = gdf.copy()

In [ ]:
gdf_01.loc[lisa.p_sim < 0.1, "cluster"] = lisa.q[lisa.p_sim < 0.1]
gdf_01["cluster"] = gdf_01["cluster"].fillna(0)
gdf_01["cluster"] = gdf_01["cluster"].map(
    {
        0: "Not significant",
        1: "High-high",
        2: "Low-high",
        3: "Low-low",
        4: "High-low",
    }
)
gdf_01.head()

In [ ]:
from splot.esda import lisa_cluster, moran_scatterplot

_ = lisa_cluster(lisa, gdf_01)

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))
gdf_01.loc[gdf_01["cluster"] == "Not significant"].plot(ax=ax, color="lightgrey")
gdf_01.loc[(gdf_01["cluster"] == "High-high")].plot(ax=ax, color="#d7191c")
gdf_01.loc[(gdf_01["cluster"] == "Low-low")].plot(ax=ax, color="#2c7bb6")
gdf_01.loc[(gdf_01["cluster"] == "Low-high")].plot(ax=ax, color="#abd9e9")
gdf_01.loc[(gdf_01["cluster"] == "High-low")].plot(ax=ax, color="#fdae61");

In [ ]:
gdf_01.explore(
    "cluster",
    prefer_canvas=True,
    cmap=["#d7191c", "#fdae61", "#abd9e9", "#2c7bb6", "lightgrey"],
)

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))

moran_scatterplot(lisa, p=0.1, scatter_kwds={"s": 5, "alpha": 0.2}, ax=ax)
plt.show()

## 0.05 Significance

In [ ]:
gdf_005 = gdf.copy()

gdf_005.loc[lisa.p_sim < 0.05, "cluster"] = lisa.q[lisa.p_sim < 0.05]
gdf_005["cluster"] = gdf_005["cluster"].fillna(0)
gdf_005["cluster"] = gdf_005["cluster"].map(
    {
        0: "Not significant",
        1: "High-high",
        2: "Low-high",
        3: "Low-low",
        4: "High-low",
    }
)
gdf_005.head()

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))
gdf_005.loc[gdf_005["cluster"] == "Not significant"].plot(ax=ax, color="lightgrey")
gdf_005.loc[(gdf_005["cluster"] == "High-high")].plot(ax=ax, color="#d7191c")
gdf_005.loc[(gdf_005["cluster"] == "Low-low")].plot(ax=ax, color="#2c7bb6")
gdf_005.loc[(gdf_005["cluster"] == "Low-high")].plot(ax=ax, color="#abd9e9")
gdf_005.loc[(gdf_005["cluster"] == "High-low")].plot(ax=ax, color="#fdae61");

In [ ]:
gdf_005.explore(
    "cluster",
    prefer_canvas=True,
    cmap=["#d7191c", "#fdae61", "#abd9e9", "#2c7bb6", "lightgrey"],
)

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))

moran_scatterplot(lisa, p=0.05, scatter_kwds={"s": 5, "alpha": 0.2}, ax=ax)
plt.show()

## 0.01 Significance

In [ ]:
gdf_001 = gdf.copy()

gdf_001.loc[lisa.p_sim < 0.01, "cluster"] = lisa.q[lisa.p_sim < 0.01]
gdf_001["cluster"] = gdf_001["cluster"].fillna(0)
gdf_001["cluster"] = gdf_001["cluster"].map(
    {
        0: "Not significant",
        1: "High-high",
        2: "Low-high",
        3: "Low-low",
        4: "High-low",
    }
)
gdf_001.head()

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))
gdf_001.loc[gdf_001["cluster"] == "Not significant"].plot(ax=ax, color="lightgrey")
gdf_001.loc[(gdf_001["cluster"] == "High-high")].plot(ax=ax, color="#d7191c")
gdf_001.loc[(gdf_001["cluster"] == "Low-low")].plot(ax=ax, color="#2c7bb6")
gdf_001.loc[(gdf_001["cluster"] == "Low-high")].plot(ax=ax, color="#abd9e9")
gdf_001.loc[(gdf_001["cluster"] == "High-low")].plot(ax=ax, color="#fdae61");

In [ ]:
gdf_001.explore(
    "cluster",
    prefer_canvas=True,
    cmap=["#d7191c", "#fdae61", "#abd9e9", "#2c7bb6", "lightgrey"],
)

In [ ]:
f, ax = plt.subplots(1, figsize=(10, 10))

moran_scatterplot(lisa, p=0.01, scatter_kwds={"s": 5, "alpha": 0.2}, ax=ax)

plt.show()

In [ ]:
import folium

In [ ]:
m = gdf_001.explore(
    "cluster",
    prefer_canvas=True,
    cmap=["#d7191c", "#fdae61", "#abd9e9", "#2c7bb6", "lightgrey"],
    style_kwds={"fillOpacity": 0.3, "weight": 0},
    name="0.01",
)

gdf_005.explore(
    "cluster",
    prefer_canvas=True,
    cmap=["#d7191c", "#fdae61", "#abd9e9", "#2c7bb6", "lightgrey"],
    style_kwds={"fillOpacity": 0.3, "weight": 0},
    name="0.05",
    m=m,
)

gdf_01.explore(
    "cluster",
    prefer_canvas=True,
    cmap=["#d7191c", "#fdae61", "#abd9e9", "#2c7bb6", "lightgrey"],
    style_kwds={"fillOpacity": 0.3, "weight": 0},
    name="0.1",
    m=m,
)

# Add a layer control to toggle the layers on and off
folium.LayerControl().add_to(m)

# Display the map
m